In [1]:
import pandas as pd
import numpy as np

## Read

In [76]:
data = pd.read_table('training.txt', sep=' ', header=None)
# name label
data.rename(columns={0:'label'}, inplace=True)
# sepearte labels
label = data.iloc[:, 0]

# remove index: from data
data = data.iloc[:, 1:]
data = data.iloc[:,1:].applymap(lambda x: x[x.find(':')+1:])

In [82]:
# rename columns and merge label
data.columns = list(range(data.shape[1]))
data = data.join(label)
# label: int, other columns: str

## Decision Tree

In [ ]:
'''
1.compute the entropy for data-set
2.for every attribute/feature:
       1.calculate entropy for all categorical values
       2.take average information entropy for the current attribute
       3.calculate gain for the current attribute
3. pick the highest gain attribute.
4. Repeat until we get the tree we desired.
'''

In [164]:
# create Tree class
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None
        self.root = None
        self.depth = 0
        
    def get_depth(self):
        if self.root == None:
            return self.depth
        else:
            return self.root.get_depth()+1

In [170]:
# tree test
root = Tree()
left = Tree()
left_left = Tree()
right = Tree()

# left
root.left = left
left.root = root

# right
root.right= right
right.root = root

# left left
left.left = left_left
left_left.root = left

# test depth
left_left.get_depth()

2

In [369]:
def get_gini(data, column, feature):
    column_count = data.groupby(column)[column].count()
    class_count = data.groupby([column, 'label'])[column].count()

    # case when specific feature selected
    class_exclude = class_count.unstack().drop(index = feature).sum()
    column_exclude = column_count.drop(index=feature).sum()
    prob_exclude = class_exclude/column_exclude
    ratio_exclude = column_exclude/n
    gini_exclude = (1-np.sum((prob_exclude)**2))

    class_feature = class_count.unstack().loc[feature]
    column_feature = column_count.loc[feature].sum()
    prob_feature = class_feature/column_feature
    ratio_feature = column_feature/n
    gini_feature = (1-np.sum((prob_feature)**2))
    
    gini = ratio_exclude*gini_exclude+ratio_feature*gini_feature
    return gini

In [372]:
get_gini(data, 20, '1')

0.7479510176693738

In [375]:
%%time
# contain gini
gini_list = []

# loop over columns except labels
for column in data.iloc[:,:-1]:
    for feature in np.unique(data[column]):
        # counts of each attribute, each class(label) count
        column_count = data.groupby(column)[column].count()
        class_count = data.groupby([column, 'label'])[column].count()

        # case when specific feature selected
        class_exclude = class_count.unstack().drop(index = feature).sum()
        column_exclude = column_count.drop(index=feature).sum()
        prob_exclude = class_exclude/column_exclude
        ratio_exclude = column_exclude/n
        gini_exclude = (1-np.sum((prob_exclude)**2))

        class_feature = class_count.unstack().loc[feature]
        column_feature = column_count.loc[feature].sum()
        prob_feature = class_feature/column_feature

        ratio_feature = column_feature/n
        gini_feature = (1-np.sum((prob_feature)**2))

        gini = ratio_exclude*gini_exclude+ratio_feature*gini_feature
        gini_list.append(gini)

Wall time: 4.44 s


In [380]:
column = 71
feature = '1'
column_count = data.groupby(column)[column].count()
class_count = data.groupby([column, 'label'])[column].count()

# case when specific feature selected
class_exclude = class_count.unstack().drop(index = feature).sum()
column_exclude = column_count.drop(index=feature).sum()
prob_exclude = class_exclude/column_exclude
ratio_exclude = column_exclude/n
gini_exclude = (1-np.sum((prob_exclude)**2))

class_feature = class_count.unstack().loc[feature]
column_feature = column_count.loc[feature].sum()
prob_feature = class_feature/column_feature
ratio_feature = column_feature/n
gini_feature = (1-np.sum((prob_feature)**2))

gini = ratio_exclude*gini_exclude+ratio_feature*gini_feature
gini

0.7290673689858093

In [387]:
data.groupby([3, 'label'])[71].count()

3  label
1  1        169
   2        128
   3        154
   4        155
2  1        405
   2        486
   3        452
   4        441
3  1        158
   2        141
   3        162
   4        149
Name: 71, dtype: int64